In [ ]:
## Next word prediction involves developing a model capable of predicting the most probable word to follow a given sequence of words in a sentence

## The task is to develop a robust Next Word Prediction model that can accurately forecast the most appropriate word following a given sequence of words.

## By analyzing the dataset inspired by Sherlock Holmes, the model should learn the linguistic patterns and relationships that govern the progression of words.

In [1]:
##Importing necessary libraries and dataset

import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense


# Read the text file
with open('/content/drive/MyDrive/Atomic_Habits_by_James_Clear.txt', 'r', encoding = 'utf-8') as file :
  text = file.read()

In [2]:
## Tokenize text to create a sequence of words

tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1

In [3]:
## Prepare the data by creating input-output pairs by splitting the text into sequences of tokens and forming n-grams from the sequences

input_sequences = []
for line in text.split('\n'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

In [4]:
## Pad input sequences to have equal length

max_sequence_len = max([len(seq) for seq in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

In [5]:
## Split sequences into input and output

X = input_sequences[:, :-1]   ## Contains all the tokens in each sequence except for the last one, representing the input context
y = input_sequences[:, -1]    ## The values of the last column in the ‘input_sequences’ array, which represents the target or predicted word

## Convert output to one-hot encode vectors

y = np.array(tf.keras.utils.to_categorical(y, num_classes=total_words))    ## Each target word is represented as a binary vector

In [6]:
## Model Training - Neural Network Architecture

model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(LSTM(150))
model.add(Dense(total_words, activation='softmax'))
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 100)           899300    
                                                                 
 lstm (LSTM)                 (None, 150)               150600    
                                                                 
 dense (Dense)               (None, 8993)              1357943   
                                                                 
Total params: 2,407,843
Trainable params: 2,407,843
Non-trainable params: 0
_________________________________________________________________
None


In [8]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=100, verbose=1)

Epoch 1/100
3010/3010 [==============================] - 128s 42ms/step - loss: 6.2531 - accuracy: 0.0745
Epoch 2/100
3010/3010 [==============================] - 124s 41ms/step - loss: 5.5398 - accuracy: 0.1215
Epoch 3/100
3010/3010 [==============================] - 122s 40ms/step - loss: 5.1484 - accuracy: 0.1458
Epoch 4/100
3010/3010 [==============================] - 118s 39ms/step - loss: 4.8176 - accuracy: 0.1636
Epoch 5/100
3010/3010 [==============================] - 118s 39ms/step - loss: 4.5164 - accuracy: 0.1808
Epoch 6/100
3010/3010 [==============================] - 119s 40ms/step - loss: 4.2325 - accuracy: 0.2013
Epoch 7/100
3010/3010 [==============================] - 119s 39ms/step - loss: 3.9600 - accuracy: 0.2267
Epoch 8/100
3010/3010 [==============================] - 119s 40ms/step - loss: 3.7003 - accuracy: 0.2554
Epoch 9/100
3010/3010 [==============================] - 121s 40ms/step - loss: 3.4564 - accuracy: 0.2886
Epoch 10/100
3010/3010 [======================

In [15]:
seed_text = "Can we talk instead of"
next_words = 4

for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted = np.argmax(model.predict(token_list), axis=-1)
    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == predicted:
            output_word = word
            break
    seed_text += " " + output_word

print(seed_text)

1/1 [==============================] - 0s 56ms/step
Can we talk instead of reportedly biobehavioral family afford
